In [1]:
import numpy as np
import medpy.io as medio
import os
from glob import glob
import pandas as pd
import sys
from util import *
from sklearn.feature_extraction.image import extract_patches
from multiprocessing import Pool
from sklearn.cross_validation import StratifiedKFold, KFold

MULTI = 8
time_point = 6
datadir_train = '../input/Training/training_'
GT_list = sorted(glob(os.path.join(datadir_train+'*', 'GT01.nii.gz')))

In [2]:
GT_list
#density
density = np.zeros(len(GT_list))
label = np.array([])
for idx, gt in enumerate(GT_list):
    test = medio.load(gt)[0]
    label = np.concatenate( (label, np.unique(np.array(test))), axis=0 )
    density[idx] = np.mean(test).astype(np.float32)


In [3]:
file_list = [str.split(f,'_')[1] for f in sorted(glob(os.path.join(datadir_train+'*')))]
GT_order = density.argsort().argsort() #order
GT_class = GT_order//5
np.transpose(np.vstack((file_list, density, GT_order, GT_class)))

array([['1', '0.000301249529002', '2', '0'],
       ['10', '0.00047363282647', '4', '0'],
       ['11', '0.00150624767412', '11', '2'],
       ['12', '0.000840928812977', '10', '2'],
       ['13', '0.00753720616922', '21', '4'],
       ['14', '0.0152339935303', '26', '5'],
       ['15', '0.000302734377328', '3', '0'],
       ['16', '0.00361569714732', '16', '3'],
       ['18', '0.0305361393839', '28', '5'],
       ['19', '0.00080744427396', '7', '1'],
       ['2', '6.8530702265e-05', '0', '0'],
       ['20', '0.00223144539632', '14', '2'],
       ['21', '0.000819512992166', '8', '1'],
       ['22', '0.0103162126616', '23', '4'],
       ['23', '0.00176111853216', '12', '2'],
       ['24', '0.0135485334322', '25', '5'],
       ['26', '0.00590960169211', '19', '3'],
       ['27', '0.0123312436044', '24', '4'],
       ['28', '0.000643807870802', '5', '1'],
       ['30', '0.00189977569971', '13', '2'],
       ['31', '0.00460792845115', '18', '3'],
       ['32', '0.00922218617052', '22', '4'

In [4]:
sss = StratifiedKFold(GT_class, n_folds=5, shuffle=True, random_state=1004)
for tr, te in sss:
    print(tr)
    print(te, [GT_class[t] for t in te])

[ 1  3  4  5  6  7 10 11 12 13 14 15 17 18 19 20 21 22 23 24 26 27 28 29]
(array([ 0,  2,  8,  9, 16, 25]), [0, 2, 5, 1, 3, 4])
[ 0  1  2  3  4  5  6  7  8  9 10 11 15 16 17 18 19 21 22 23 25 27 28 29]
(array([12, 13, 14, 20, 24, 26]), [1, 4, 2, 3, 5, 0])
[ 0  2  3  4  6  7  8  9 10 12 13 14 15 16 17 19 20 23 24 25 26 27 28 29]
(array([ 1,  5, 11, 18, 21, 22]), [0, 5, 2, 1, 4, 3])
[ 0  1  2  4  5  6  7  8  9 11 12 13 14 16 18 19 20 21 22 23 24 25 26 27]
(array([ 3, 10, 15, 17, 28, 29]), [2, 0, 5, 4, 3, 1])
[ 0  1  2  3  5  8  9 10 11 12 13 14 15 16 17 18 20 21 22 24 25 26 28 29]
(array([ 4,  6,  7, 19, 23, 27]), [4, 0, 3, 2, 1, 5])


In [5]:
#np.save(arr = GT_class, file = '../input/classLabel_5_folds.npy')

In [6]:
np.load('../input/classLabel_5_folds.npy')

array([0, 0, 2, 2, 4, 5, 0, 3, 5, 1, 0, 2, 1, 4, 2, 5, 3, 4, 1, 2, 3, 4, 3,
       1, 5, 4, 0, 5, 3, 1])

In [7]:
GT_class

array([0, 0, 2, 2, 4, 5, 0, 3, 5, 1, 0, 2, 1, 4, 2, 5, 3, 4, 1, 2, 3, 4, 3,
       1, 5, 4, 0, 5, 3, 1])

In [8]:
from data_multiscale import make_data_list
from util import make_range
from matplotlib import pyplot as plt
%matplotlib inline

# Visualize all the data

In [12]:
img_list, label_list = make_data_list(range(10), isTrainOrVal = True, datadir = datadir_train)
w, c = 256, 32

In [ ]:
for j in xrange(30):
    plt.figure(figsize=(8,128))
    for i in xrange(c):
        plt.subplot(c,2,(2*i+1))
        plt.imshow(img_list[j][0][:,:,i], cmap=plt.cm.gray)
        plt.title('modality')
        plt.subplot(c,2,(2*i+2))
        plt.imshow(label_list[j][:,:,i], cmap=plt.cm.gray)
        plt.title('outcome')
    plt.savefig('./brain/train/{}.pdf'.format(j))

In [13]:
'DONE'

'DONE'

# GT_list_train check

In [ ]:
k=0
row_size=32
channel_size=4
num_patch=200
proportion=0.5

In [ ]:
test_sum = 0
Y = []
X = []
X_zoom= []
GT_list=[]
label = label_list[k]
w, _, c =label.shape

m1 = np.where(np.sum(label, axis=1)>0)[0]
m2 = np.where(np.sum(label, axis=0)>0)[0]
m3 = np.where(np.sum(label, axis=1)>0)[1]
patch_dim = (row_size ** 2)*channel_size

N_GT_patches = int(num_patch*(proportion))
N_random_patches = num_patch-N_GT_patches
for ind in zip( np.random.choice(range(make_range(min(m1), w, row_size), \
    make_range(max(m1), w, row_size, False)), N_GT_patches), \
                np.random.choice(range(make_range(min(m2), w, row_size), \
                    make_range(max(m2), w, row_size, False)), N_GT_patches), \
                np.random.choice(range(make_range(min(m3), c, channel_size), \
                    make_range(max(m3), c, channel_size, False)), N_GT_patches)):

    ind_list = np.array(ind) + np.array([row_size/2, row_size/2, channel_size/2]) \
    + np.array(zip((np.arange(patch_dim)/(row_size*channel_size)), \
        (np.arange(patch_dim)/channel_size)%row_size , np.arange(patch_dim)%channel_size))

    ind_list_zoom = np.array(ind) + np.array(zip((np.arange(8*patch_dim)/(4*row_size*channel_size)), \
        (np.arange(8*patch_dim)/(2*channel_size))%(2*row_size) , np.arange(8*patch_dim)%(2*channel_size)))

    ind_list = tuple(map(tuple, ind_list))
    ind_list_zoom = tuple(map(tuple, ind_list_zoom))
    y = np.array([(label_list[k])[idx] for idx in ind_list]).reshape(row_size,row_size,channel_size)
    gt = (np.sum(y) > 0)
    x = []

    x_zoom = []
    x_sag = []
    x_sag_zoom = []
    for m in xrange(time_point):
        img = np.array([(img_list[k][m])[idx] \
            for idx in ind_list]).reshape(row_size,row_size,channel_size)
        x.append(img)
        x_sag.append(transform_sag(img))
        img_zoom = np.array([(img_list[k][m])[idx] \
            for idx in ind_list_zoom]).reshape(2*row_size,2*row_size,2*channel_size)
        temp = transform(img_zoom)
        x_zoom.append(temp)
        x_sag_zoom.append(transform_sag(temp))

    Y.append(y)
    X.append(np.array(x))
    X_zoom.append(np.array(x_zoom))
    GT_list.append(gt)

    Y.append(transform_sag(y))
    X.append(np.array(x_sag))
    X_zoom.append(np.array(x_sag_zoom))
    GT_list.append(gt)

for ind in zip( np.random.choice(w-2*row_size, N_random_patches), \
                    np.random.choice(w-2*row_size, N_random_patches), \
                    np.random.choice(c-2*channel_size, N_random_patches) ) :

    ind_list = np.array(ind) + np.array([row_size/2, row_size/2, channel_size/2]) \
    + np.array(zip((np.arange(patch_dim)/(row_size*channel_size)), \
        (np.arange(patch_dim)/channel_size)%row_size , np.arange(patch_dim)%channel_size))

    ind_list_zoom = np.array(ind) + np.array(zip((np.arange(8*patch_dim)/(4*row_size*channel_size)), \
        (np.arange(8*patch_dim)/(2*channel_size))%(2*row_size) , np.arange(8*patch_dim)%(2*channel_size)))

    ind_list = tuple(map(tuple, ind_list))
    ind_list_zoom = tuple(map(tuple, ind_list_zoom))
    y = np.array([(label_list[k])[idx] for idx in ind_list]).reshape(row_size,row_size,channel_size)
    gt = (np.sum(y) > 0)
    x = []
    x_zoom = []            
    x_sag = []
    x_sag_zoom = []
    for m in xrange(time_point):
        img = np.array([(img_list[k][m])[idx] \
            for idx in ind_list]).reshape(row_size,row_size,channel_size)
        x.append(img)
        x_sag.append(transform_sag(img))
        img_zoom = np.array([(img_list[k][m])[idx] \
            for idx in ind_list_zoom]).reshape(2*row_size,2*row_size,2*channel_size)
        temp = transform(img_zoom)
        x_zoom.append(temp)
        x_sag_zoom.append(transform_sag(temp))

    if np.std(img) > 1e-5:
        Y.append(y)
        X.append(np.array(x))
        X_zoom.append(np.array(x_zoom))
        GT_list.append(gt)

        Y.append(transform_sag(y))
        X.append(np.array(x_sag))
        X_zoom.append(np.array(x_sag_zoom))
        GT_list.append(gt)
    else:
        test_sum += 1
print(test_sum)        

In [ ]:
X, X_zoom, Y, GT_list = np.array(X).astype('float32'), np.array(X_zoom).astype('float32'), np.array(Y), np.array(GT_list)

In [ ]:
np.mean(GT_list), len(GT_list)

In [ ]:
GT_list-0